## Convolutional Neural Network

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

In [3]:
mnist = input_data.read_data_sets("/tmp/data", one_hot=True)

W0622 13:36:56.988272 14976 deprecation.py:323] From <ipython-input-3-5bb76f0ba894>:1: read_data_sets (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
W0622 13:36:56.998865 14976 deprecation.py:323] From C:\Users\smart\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:260: maybe_download (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated and will be removed in a future version.
Instructions for updating:
Please write your own downloading logic.
W0622 13:36:57.003757 14976 deprecation.py:323] From C:\Users\smart\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\base.py:252: _internal_retry.<locals>.wrap.<locals>.wrapped_fn (from tensorflow.contrib.learn.python.learn.datasets.base) is deprecated a

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data\train-images-idx3-ubyte.gz


W0622 13:36:58.663300 14976 deprecation.py:323] From C:\Users\smart\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:267: extract_labels (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.data to implement this functionality.
W0622 13:36:58.668993 14976 deprecation.py:323] From C:\Users\smart\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:110: dense_to_one_hot (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use tf.one_hot on tensors.


Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data\train-labels-idx1-ubyte.gz


W0622 13:36:59.013403 14976 deprecation.py:323] From C:\Users\smart\Anaconda3\envs\tensorflow_env\lib\site-packages\tensorflow\contrib\learn\python\learn\datasets\mnist.py:290: DataSet.__init__ (from tensorflow.contrib.learn.python.learn.datasets.mnist) is deprecated and will be removed in a future version.
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz


### Build model

In [15]:
#Nodes in the hidden layer
n_nodes_hl1 = 500
n_nodes_hl2 = 500
n_nodes_hl3 = 500

In [16]:
#10 digits to recognize.
n_classes = 10 

#take 100 images at one time
batch_size = 100

In [17]:
#Get placeholders of values in graph
#784 = size of single image
x = tf.placeholder('float', [None, 784])
y = tf.placeholder('float')

In [26]:
def neural_network_model(data):
     #Add weights into input 
    #Purpose of bias is to handle scenario where all neurons fire a 0 into that layer
    hidden_1_layer = {'weights': tf.Variable(tf.random_normal([784, n_nodes_hl3])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl1]))}
    hidden_2_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl1, n_nodes_hl2])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    hidden_3_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl2, n_nodes_hl3])),
                     'biases': tf.Variable(tf.random_normal([n_nodes_hl2]))}
    output_layer = {'weights': tf.Variable(tf.random_normal([n_nodes_hl3, n_classes])),
                     'biases': tf.Variable(tf.random_normal([n_classes]))}
    
    #Doing the calculations now
    #After multiplication, added biases to the object - for each layer
    l1 = tf.add(tf.matmul(data, hidden_1_layer['weights']), hidden_1_layer['biases'])
    l1 = tf.nn.relu(l1)
    
    l2 = tf.add(tf.matmul(l1, hidden_2_layer['weights']), hidden_2_layer['biases'])
    l2 = tf.nn.relu(l2)
    
    l3 = tf.add(tf.matmul(l2, hidden_3_layer['weights']), hidden_3_layer['biases'])
    l3 = tf.nn.relu(l3)
    
    output = tf.matmul(l3, output_layer['weights']) + output_layer['biases']
    
    return output

In [29]:
def train_neural_network(x):
    prediction = neural_network_model(x)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y))
    #Use Adam Optimizer to optimize cost 
    optimizer = tf.train.AdamOptimizer().minimize(cost) #can specify learning rate as param. Default is 0.001
    hm_epochs = 10
    with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        
        #For each epoch, run optimizer at cost against batch of data
        #Output cost for each epoch which should decline
        for epoch in range(hm_epochs):
            epoch_loss = 0
            for _ in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size)
                
                _, c = sess.run([optimizer, cost], feed_dict={x:epoch_x, y:epoch_y})
                epoch_loss += c
            print('Epoch ',epoch , 'completed out of ', hm_epochs, 'loss: ', epoch_loss)
        
        correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct, "float"))

        print('Accuracy', accuracy.eval({x:mnist.test.images, y:mnist.test.labels}))

In [30]:
train_neural_network(x)

Epoch  0 completed out of  10 loss:  1570343.7462768555
Epoch  1 completed out of  10 loss:  376684.6121292114
Epoch  2 completed out of  10 loss:  206226.17829036713
Epoch  3 completed out of  10 loss:  125101.78372516061
Epoch  4 completed out of  10 loss:  71221.9854708314
Epoch  5 completed out of  10 loss:  47483.68318994343
Epoch  6 completed out of  10 loss:  30210.726671077176
Epoch  7 completed out of  10 loss:  23509.05244976282
Epoch  8 completed out of  10 loss:  18143.755818714497
Epoch  9 completed out of  10 loss:  15813.990440645219
Accuracy 0.9522
